<a href="https://colab.research.google.com/github/Michwynn/London-Airbnb-Analysis---2/blob/Elias/Airbnb_Data_Base_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Configuration and library set-up**

In [25]:
# data manipulation
import pandas as pd
import numpy as np 
import re # regex
import random
from collections import Counter, defaultdict

# machine learning
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# timeit
from tqdm import tqdm

# data visualisation
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# set up working directory
import os
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Airbnb_Milestone2

# supress warnings
import warnings 
warnings.filterwarnings('ignore')

# Display all columns
pd.set_option('display.max_columns', None)

# NLP
!pip install pycld2
import pycld2 as cld2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1wUOfFY-ki2nFzneeaTtXLEeMjaSdKrrj/Airbnb_Milestone2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41.4 MB 1.2 MB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834319 sha256=7dd369dd1cdd1a60b8282c4e13229aab7ca25b74cbbdeda1a58cb0b178ad910c
  Stored in directory: /root/.cache/pip/wheels/ed/e4/58/ed2e9f43c07d617cc81fe7aff0fc6e42b16c9cf6afe960b614
Successfully built pycld2


In [2]:
# read data
reviews_df = pd.read_csv('Datasets/reviews.csv') 
# remove white spaces in column headings
reviews_df.columns = reviews_df.columns.str.strip() 
# Examine dataset
display(reviews_df.head(10))
display(reviews_df.shape)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi..."
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator… he was very hel...
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.<br/>Good l...
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host
5,52228441,488275073602903479,2021-11-04,37185586,Ruslans,The flat is very well connected to the city by...
6,17501535,668631693343526023,2022-07-11,5132271,Marc,Nice place
7,52228441,588257361951713358,2022-03-22,248672143,Sveinung,Big apartment but a little bit run down. Kitc...
8,17501535,633193507110858827,2022-05-23,413544004,Daniel,"Amazing place, we loved it. Tons of space, nic..."
9,52228441,491866989310660615,2021-11-09,8452946,Johnny,"Duccio was a great host, super friendly with l..."


(1216212, 6)

# **Data Cleaning**

**Steps to follow**

1.   Drop rows containing NAN values (DONE)
2.   Drop rows with empty string for 'comments' column (DONE)
3.   Remove HTML tags from string in 'comments' column (DONE)
4.   Remove special characters including emojis from strings (DONE)
5.   Drop rows containing non-english string in 'comments' column (DONE)
6.   Remove stopwords from string in 'comments' column
7.   Remove any short-word form string in 'comments' column






In [3]:
# check if there is any null value 
reviews_df.isnull().sum()

listing_id        0
id                0
date              0
reviewer_id       0
reviewer_name     2
comments         92
dtype: int64

In [4]:
# check if there is any empty string in the comments column
reviews_df.loc[reviews_df["comments"] == "."]

,listing_id,id,date,reviewer_id,reviewer_name,comments
92,21526298,675869744229404323,2022-07-21,215666012,David,.
319,32537244,572622781,2019-12-03,310625523,Fernando,.
2388,3370968,551274991374749641,2022-01-30,88466671,Bekee,.
3143,13396482,562849095,2019-11-11,299309771,Cecilie,.
3334,41509,49170962,2015-10-01,736090,Alberto,.
...,...,...,...,...,...,...
1209963,31495400,518304382,2019-08-26,34520842,Augusto,.
1212428,31201633,419606016,2019-03-04,243989008,Javier,.
1212852,31304270,419096711,2019-03-03,244943897,Bernard,.
1214589,32002019,474417589,2019-06-23,64631352,Margaret,.


In [5]:
# Drop the null value and comments value with "."
reviews_df = reviews_df[reviews_df['comments'].notna()]
display(reviews_df.shape)

(1216120, 6)

---
92 Comments containing null value dropped

In [58]:
# Drop the rows with empty string for comments column
reviews_df = reviews_df.loc[reviews_df["comments"] != "."]
display(reviews_df.shape)

(1214643, 6)

---
1477 Comments containing empty string dropped

In [7]:
# helper function to remove HTML tags from comments
from bs4 import BeautifulSoup
copy_df = reviews_df.copy()
#small_df = copy_df.iloc[:100,]
copy_df['comments'] = copy_df['comments'].apply(lambda s: BeautifulSoup(s).text)
display(copy_df.head(100))

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi..."
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator… he was very hel...
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.Good locati...
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host
...,...,...,...,...,...,...
96,17501535,588209695468203799,2022-03-22,31576937,Julian,"Superb hospitality. Dominic was friendly, resp..."
97,32537244,617521403,2020-03-13,84592669,Bun-Hok,"Good place to stay, although a bit noisy, as i..."
98,17501535,594341871,2020-01-18,84447796,Mallory,Easy and convenient check-in/check-out. The s...
99,17501535,561415884152066004,2022-02-13,147957618,Sandra,Dominic was a great host! We had to reach out ...


In [8]:
# removing special characters including punctuations and emojis. 
copy2_df = copy_df.copy()

copy2_df['comments'] = copy2_df['comments'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
copy2_df.columns = copy2_df.columns.str.strip() 

display(copy2_df)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi..."
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator he was very help...
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.Good locati...
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host
...,...,...,...,...,...,...
1216207,14832630,184884203,2017-08-20,54407484,丽云,干净舒适适合家庭入住地段优越景点全部很多都可以步行到达节省了交通费房间设施齐备能满足我们的一...
1216208,14832630,173531056,2017-07-23,130284505,Véronique,Excellente situation au plein centre de Londre...
1216209,14832630,153170957,2017-05-20,119296298,Nicole,"Todo perfecto Ubicación, comodidad y Rachel c..."
1216210,14832630,145456398,2017-04-18,16394435,Christophe,Tout dabord même si nous navons pas eu le plai...


# **Comments Language Identification**

In [9]:
# installing and Importing fasttext library for language identification
!pip install fasttext
import fasttext
# settting up the pretrained model
fasttxt_model = fasttext.load_model("lid.176.bin")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.8 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3165333 sha256=def199d19678154661ef36dc3ee0530ae867f847834aa273e95ace2c5e8c4190
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [10]:
# test model with example text
fasttxt_model.predict('이 집은 정말 좋아요! 맘에 쏙 드네요')[0][0][-2:]

'ko'

In [11]:
# Adding an extra column for language identification
copy2_df['lang'] = copy2_df['comments'].apply(lambda x: fasttxt_model.predict(x)[0][0][-2:])
display(copy2_df)

,listing_id,id,date,reviewer_id,reviewer_name,comments,lang
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi...",en
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...,en
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator he was very help...,en
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.Good locati...,en
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host,en
...,...,...,...,...,...,...,...
1216207,14832630,184884203,2017-08-20,54407484,丽云,干净舒适适合家庭入住地段优越景点全部很多都可以步行到达节省了交通费房间设施齐备能满足我们的一...,en
1216208,14832630,173531056,2017-07-23,130284505,Véronique,Excellente situation au plein centre de Londre...,fr
1216209,14832630,153170957,2017-05-20,119296298,Nicole,"Todo perfecto Ubicación, comodidad y Rachel c...",es
1216210,14832630,145456398,2017-04-18,16394435,Christophe,Tout dabord même si nous navons pas eu le plai...,fr


In [12]:
# calculating number of rows with non-english comments 
display(copy2_df.lang.value_counts())
prop = (1214643-1089747)/1214643*100
f'The total non-english comments proportion is {prop}.'

en    1089747
fr      45182
es      22556
de      16384
it      10215
       ...   
br          1
mk          1
ne          1
io          1
ms          1
Name: lang, Length: 77, dtype: int64

'The total non-english comments proportion is 10.282527458685392.'

---
**The total non-english comments proportion is 10.28%. 
Therefore, it is safe to drop the rows with non-english comments.**




In [13]:
# keeping only rows with English comments
copy3_df = copy2_df[copy2_df['lang']=='en']
display(copy3_df)

,listing_id,id,date,reviewer_id,reviewer_name,comments,lang
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi...",en
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...,en
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator he was very help...,en
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.Good locati...,en
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host,en
...,...,...,...,...,...,...,...
1216194,14832630,107089577,2016-10-09,46576370,Anders,"This was an excellent apartment. Very clean, w...",en
1216195,14832630,105494942,2016-10-02,31582062,Magda,The place was a dream come true\r Its newly re...,en
1216196,14832630,104922701,2016-09-29,95543679,Joe,Her place is so greatNice and clean. My wife a...,en
1216197,14832630,102192086,2016-09-17,16267738,Neta,Right at the heart of Soho.The place is comfor...,en


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [41]:
# Special case that has both chinese and English
display(copy3_df.loc[[1216207]])

,listing_id,id,date,reviewer_id,reviewer_name,comments,lang
1216207,14832630,184884203,2017-08-20,54407484,丽云,干净舒适适合家庭入住地段优越景点全部很多都可以步行到达节省了交通费房间设施齐备能满足我们的一...,en


In [75]:
copy3_df['Deteced_lang'] = copy3_df['comments'].map(lambda a: cld2.detect(a,bestEffort=True)[2][1][0])
#test1 = '干净舒适适合家庭入住地段优越景点全部很多都可以步行到达节省了交通费房间设施齐备能满足我们的一切需求房东十分热情周到联系及时入住期间有个灯坏了跟房东联系后当天出游回来已经完美解决完全没有影响我们的行程Clean and comfortable, suitable for family occupancy, lots of advantages, many attractions can be reached on foot, saving traffic costs. The room is well equipped. Can meet all our needs. Landlord very warm and thoughtful, timely contact. Check in, there is a lamp broken, and after contact with the landlord, the day of travel back, has been perfect solution. It didnt affect our schedule at all.'
#test2 = 'No cumple del todo con las fotos. Buena ubicación pero muy básica y antigua. La ducha tampoco funcionaba correctamente. This is not acceptable. I hate it'
#print(cld2.detect(test1)[2][0][0],cld2.detect(test1)[2][1][0])
#print(cld2.detect(test2))

In [85]:
# rows containing multipel languages
copy3_df['Deteced_lang'] = copy3_df['comments'].map(lambda a: cld2.detect(a,bestEffort=True)[2][1][0])
copy4_df = copy3_df[copy3_df['Deteced_lang']!='Unknown']
copy4_df.comments.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
display(copy4_df)


,listing_id,id,date,reviewer_id,reviewer_name,comments,lang,Deteced_lang
3289,37641803,550704790,2019-10-19,291254723,Jihyun,Everything was great.,en,ENGLISH
4808,29225347,349944474,2018-11-18,198843912,Mako,AirbnbIt is lucky that my first Airbnb was at ...,en,ENGLISH
5016,34593988,541323918,2019-10-05,295766581,刘业,Dominic is very warm. And the house is very cl...,en,Chinese
7979,11539785,337121568,2018-10-15,218872556,芸,Niconice5Nico is very nice and kindHis home is...,en,Chinese
9427,42010,551480413,2019-10-20,296871133,Sihyun,Its really spacious and cozy home with a lovel...,en,Korean
...,...,...,...,...,...,...,...,...
1212119,36710362,579332126,2019-12-20,145329501,Ivy,"zone1A student studying in the UK, traveling t...",en,Chinese
1212725,31856332,557174927,2019-10-31,299704917,Market,"Ubytovani presne odpovida nabidce. Cisty, svet...",en,CZECH
1212937,36904055,586593895,2020-01-02,115588137,铭铭,"living environment, facility, and so on are al...",en,Chinese
1213913,36660,457086421,2019-05-23,68239245,雅婷,The house is big and beautiful. The bed is sof...,en,ChineseT


In [87]:
display(copy3_df[copy3_df['Deteced_lang']!='Unknown'])

,listing_id,id,date,reviewer_id,reviewer_name,comments,lang,Deteced_lang
3289,37641803,550704790,2019-10-19,291254723,Jihyun,Everything was great. 좋은 집이었어요,en,ENGLISH
4808,29225347,349944474,2018-11-18,198843912,Mako,初めてAirbnbを利用したのがﾗｹｯｼｭさんのお宅で良かったです彼に会うまでは緊張していた...,en,ENGLISH
5016,34593988,541323918,2019-10-05,295766581,刘业,房东特别热心房屋也非常的干净整洁温馨Dominic is very warm. And th...,en,Chinese
7979,11539785,337121568,2018-10-15,218872556,芸,房东Nico人很nice家里装修很好一应俱全乐于分享位置离地铁站仅5分钟出行很方便来伦敦玩儿...,en,Chinese
9427,42010,551480413,2019-10-20,296871133,Sihyun,Its really spacious and cozy home with a lovel...,en,Korean
...,...,...,...,...,...,...,...,...
1212119,36710362,579332126,2019-12-20,145329501,Ivy,一位在英留学生圣诞假期去伦敦旅游选中这个地方居住性价比超高而且在伦敦zone1离地铁站很近附...,en,Chinese
1212725,31856332,557174927,2019-10-31,299704917,Market,"Ubytovani presne odpovida nabidce. Cisty, svet...",en,CZECH
1212937,36904055,586593895,2020-01-02,115588137,铭铭,住宿环境设施等超乎预期房东人特别好出门就是各种居家超市门口就有便利的交通设施小屋子特别舒适做...,en,Chinese
1213913,36660,457086421,2019-05-23,68239245,雅婷,The house is big and beautiful. The bed is sof...,en,ChineseT


In [79]:
# rows containing non-meaningful words
copy3_df['Deteced_lang'] = copy3_df['comments'].map(lambda a: cld2.detect(a,bestEffort=True)[2][0][0])
display(copy3_df[copy3_df['Deteced_lang']=='Unknown'])

,listing_id,id,date,reviewer_id,reviewer_name,comments,lang,Deteced_lang
2108,628794829558861632,652703934422703971,2022-06-19,377846463,Akosua,,en,Unknown
4610,23497272,675891212634955036,2022-07-21,406862005,Chu,,en,Unknown
9728,37864812,638892960638599887,2022-05-31,178100666,Danny,,en,Unknown
11646,30143771,637533908934440948,2022-05-29,456961608,Viktor,,en,Unknown
11851,37357378,620097889542864989,2022-05-05,394924014,Siti,,en,Unknown
...,...,...,...,...,...,...,...,...
1203517,29847473,368119968,2019-01-06,62460688,Neil,,en,Unknown
1204111,30785525,673793309,2020-10-12,279782355,El,,en,Unknown
1205106,30576739,556714263,2019-10-30,189330712,Carina,,en,Unknown
1208985,31322762,405443376,2019-01-27,237030791,James,,en,Unknown
